# final.ipynb: 通过已经训练的决策树，最终解题
前提：跑完`train.ipynb`或者`model.ipynb`，最好跑完`waveform.py`。如果没有跑`waveform.py`，请将下面标注出来的代码块取消注释。

In [3]:
import multiprocessing
import warnings
import numpy as np
import h5py
from utils import loadData, getNum, getPePerWF, saveans, lossfunc_eval, lossfunc_train
import lightgbm as lgb
from tqdm import tqdm

读取测试集

In [4]:
testpath = "data/final.h5"
testWF = loadData(testpath, 'test')

  0%|          | 0/16 [00:00<?, ?it/s]

正在读取data


100%|██████████| 16/16 [01:15<00:00,  4.70s/it]


如果已经跑完了`waveform.py`，跑上面的代码块；如果没有，跑下面的代码块。

总之，得到第一个决策树需要的那些参数，并为第二个决策树做准备。

In [5]:
# 如果跑了waveform.py，请运行下面的代码块，否则不要运行
with h5py.File('./train/final_wf.h5', 'r') as ipt:
    intTestWF = ipt['Waveform']['intWF'][...]
    pointsPerTestWF = ipt['Waveform']['pointsPerWF'][...]
    pePerTestWFCalc = ipt['Waveform']['pePerWFCalc'][...]
    meanPeTimePerTestWF = ipt['Waveform']['meanPeTimePerWF'][...]
    wfIndices = ipt['WfIndices'][...]

In [ ]:
# 如果没有跑waveform.py，请将这个代码块取消注释，运行这个代码块
'''
numPEW, wfIndices = getNum(testWF)

print("正在去除波形噪音...")
denoisedTestWF = np.empty((testWF.shape[0], 1000), dtype='<i2')
step = 3000000
for i in tqdm(range(testWF.shape[0] // step + 1)):
    denoisedTestWF[step*i:step*(i+1)] = np.maximum(918-testWF['Waveform'][step*i:step*(i+1), :], 0)
print("正在做波形积分...")
intTestWF = np.sum(denoisedTestWF, axis=1)
print("正在计算超出阈值的点数...")
pointsPerTestWF = np.sum(denoisedTestWF > 0, axis=1)

chunkNum = 100
splitedDenoisedTestWF = np.array_split(denoisedTestWF, chunkNum)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    with multiprocessing.Pool(8) as pool:
        res = np.concatenate(
            list(
                tqdm(
                    pool.imap(
                        getPePerWF,
                        splitedDenoisedTestWF
                    ),
                    total=chunkNum
                )
            ),
            axis=1
        )
    pePerTestWFCalc, meanPeTimePerTestWF = res
'''

In [6]:
pePerWF = pePerTestWFCalc

运算得到决策树所需要的五个feature

In [7]:
print("正在计算PE总数，平均值与标准差...")

splitPePerTestWFFinal = np.split(pePerWF, wfIndices[1:-1])
peTotal = np.empty(4000)
peMean = np.empty(4000)
peStd = np.empty(4000)
for index, pePerTestWFFinalChunk in enumerate(tqdm(splitPePerTestWFFinal)):
    peTotal[index] = np.sum(pePerTestWFFinalChunk)
    peMean[index] = np.mean(pePerTestWFFinalChunk)
    peStd[index] = np.std(pePerTestWFFinalChunk)

print("正在计算PETime平均值与标准差...")
splitMeanPeTimePerTestWF = np.split(meanPeTimePerTestWF, wfIndices[1:-1])
peTimeMean = np.empty(4000)
peTimeStd = np.empty(4000)
for index, meanPeTimePerTestWFFinalChunk in enumerate(tqdm(splitMeanPeTimePerTestWF)):
    peTimeMean[index] = np.nanmean(meanPeTimePerTestWFFinalChunk)
    peTimeStd[index] = np.nanstd(meanPeTimePerTestWFFinalChunk)

100%|██████████| 4000/4000 [00:01<00:00, 3395.66it/s]


喂进决策树，得到最终答案动量p

In [8]:
gbmForP = lgb.Booster(model_file='./model/modelPCalc.txt')
answerP = gbmForP.predict(
    np.stack(
        (peTotal, peMean, peStd, peTimeMean, peTimeStd),
        axis=1
    )
)

将答案存为标准格式，完成！

In [9]:
saveans(answerP, './ans/ans23.h5')